<a href="https://colab.research.google.com/github/NLPiation/tutorial_notebooks/blob/main/summarization/hf_BART_inference_breakdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Sample code to show how we generate a sentence using seq2seq architecture!
The code is the supplementary material to the story published in NLPiation medium. Follow [the link](https://pub.towardsai.net/a-full-introduction-on-text-summarization-using-deep-learning-with-sample-code-ft-huggingface-d21e0336f50c) for a detailed explanation of the encoder-decoder architecture and code.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NLPiation/tutorial_notebooks/blob/main/summarization/hf_BART_inference_breakdown.ipynb)

## Installing HuggingFace/Importing

Start by installing the Transformers library (by Huggingface) and then import the modules.

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 3.9 MB/s 
     |████████████████████████████████| 61 kB 243 kB/s 
     |████████████████████████████████| 596 kB 43.1 MB/s 
     |████████████████████████████████| 895 kB 19.9 MB/s 
     |████████████████████████████████| 3.3 MB 27.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

## Load the Model

We should put the model in evaluation mode to get a consistent result after each run and avoid the randomness of layers like dropout. Since the huggingface library load the models in eval() mode by default it is not necesary here.

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
# model = model.eval()

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

## Get the Necessary Components

Let's first call the model object to see all the sub-modules and layers. We can find what each layer is called and how many layers are in the network.

In [ ]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
   

Now, we can use the built-in function to get each componont. The language model head (lm_head) is also necesary as it will generates the last probability output.

In [ ]:
the_encoder = model.get_encoder()
the_decoder = model.get_decoder()
last_linear_layer = model.lm_head

## Tokenize a Sample Article

In [ ]:
ARTICLE_TO_SUMMARIZE = """sebastian vettel is determined to ensure the return of a long-standing ritual at ferrari is not a one-off this season. fresh from ferrari's first victory in 35 grands prix in malaysia 11 days ago, and ending his own 20-race drought, vettel returned to a hero's welcome at the team's factory at maranello last week. the win allowed ferrari to revive a tradition not seen at their base for almost two years since their previous triumph in may 2013 at the spanish grand prix courtesy of fernando alonso. sebastian vettel reflected on his stunning win for ferrari at the malaysian grand prix during the press conference before the weekend's chinese grand prix in shanghai the four-time world champion shares a friendly discussion with mclaren star jenson button four-times world champion vettel said: 'it was a great victory we had in malaysia, great for us as a team, and for myself a very emotional day - my first win with ferrari. 'when i returned to the factory on wednesday, to see all the people there was quite special. there are a lot of people working there and as you can imagine they were very, very happy. 'the team hadn't won for quite a while, so they enjoyed the fact they had something to celebrate. there were a couple of rituals involved, so it was nice for them to get that feeling again.' asked as to the specific nature of the rituals, vettel replied: 'i was supposed to be there for simulator work anyway, but it was quite nice to receive the welcome after the win. ferrari's vettel and britta roeske arrive at the shanghai circuit along with a ferrari mechanic, vettel caught up with members of his old team red bull on thursday 'all the factory got together for a quick lunch. it was quite nice to have all the people together in one room - it was a big room! - so we were able to celebrate altogether for a bit. 'i also learned when you win with ferrari, at the entry gate, they raise a ferrari flag - and obviously it's been a long time since they last did that. 'some 10 years ago there were a lot of flags, especially at the end of a season, so this flag will stay there for the rest of the year. 'we will, of course, try and put up another one sometime soon.' inside the ferrari garage, vettel shares a discussion with team staff as he looks to build on his sepang win ferrari team principal maurizio arrivabene shares a conversation with vettel at the team's hospitality suite the feeling is that will not happen after this weekend's race in china as the conditions at the shanghai international circuit are expected to suit rivals mercedes. not that vettel believes his success will be a one-off, adding: 'for here and the next races, we should be able to confirm we have a strong package and a strong car. 'we will want to make sure we stay ahead of the people we were ahead of in the first couple of races, but obviously knowing mercedes are in a very, very strong position. 'in general, for the start of a season things can be up and down, and we want to make sure there is quite a lot of ups, not so many downs. 'but it's normal in some races you are more competitive than others. 'we managed to do a very good job in malaysia, but for here and the next races we have to be realistic about we want to achieve.' ferrari mechanics show their joy after vettel won the malaysian grand prix, helping record the team's first formula one win since 2013 at the spanish grand prix"""

In [ ]:
tokenized_input = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, truncation=True, return_tensors='pt')

## Encoding the Article

We pass both token_ids and the attention_mask to the model that are generated by the tokenizer.

In [ ]:
input_representation = the_encoder(input_ids = tokenized_input.input_ids,
                                   attention_mask = tokenized_input.attention_mask)
print( input_representation )

BaseModelOutput(last_hidden_state=tensor([[[ 0.0076,  0.0438,  0.0330,  ...,  0.0049,  0.0072,  0.0013],
         [ 0.2069, -0.5512, -0.0667,  ..., -0.2238, -0.1640,  0.0192],
         [ 0.1963, -0.5009,  0.3967,  ...,  0.0317,  0.0772,  0.1804],
         ...,
         [-0.1127, -0.2903, -0.0103,  ...,  0.0528, -0.1473, -0.0085],
         [ 0.0686, -0.0143, -0.0834,  ..., -0.0261, -0.1815,  0.3171],
         [ 0.0344,  0.4228,  0.0438,  ...,  0.0531,  0.0063, -0.0064]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


## Decoding the Representation

Now, we use the encoded representation of the input alongside with the start token id to the decoder to predict the first token of the summary. Lastly, pass the decoder's output to the linear layer to get the probability of each token in the vocabulary to be the next predicted token.

In [ ]:
start_token = torch.tensor( [tokenizer.bos_token_id] ).unsqueeze(0)
mask_ids    = torch.tensor( [1] ).unsqueeze(0)

In [ ]:
decoder_output = the_decoder(input_ids=start_token,
                             attention_mask=mask_ids,
                             encoder_hidden_states=input_representation[0],
                             encoder_attention_mask=tokenized_input.attention_mask )

decoder_output = decoder_output.last_hidden_state

In [ ]:
logits = last_linear_layer(decoder_output[0])
predicted_token = logits.argmax(1)
print(predicted_token)

tensor([1090])


The first predicted token id is **1090**. Let's use the convert_ids_to_tokens() function to see what this id represents in the BART's vocabulary.

In [ ]:
tokenizer.convert_ids_to_tokens(predicted_token)

['se']

So, it represents 'se' 🤔! We saw how the model generate tokens to form a summary. Let's use the huggingface's generate function to actually generate a summary with more tokens and decode the IDs to readable words.

In [ ]:
summary_ids = model.generate(tokenized_input['input_ids'], max_length=15, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

['sebastian vettel won the malaysian']


In [ ]:
tokenizer.convert_ids_to_tokens( summary_ids[0] )

['</s>',
 '<s>',
 'se',
 'b',
 'ast',
 'ian',
 'Ġve',
 'tt',
 'el',
 'Ġwon',
 'Ġthe',
 'Ġmal',
 'ays',
 'ian',
 '</s>']

In [ ]:
summary_ids

tensor([[   2,    0, 1090,  428, 1988,  811, 5030, 5967,  523,  351,    5, 8196,
         4113,  811,    2]])

Interesting, so the word 'se' was the first token to predict the name 'sebastian'! Because the tokenizer break-down multi sylabus words into multiple tokens to reduce the overall vocabulary size.

Read the medium post [here](https://pub.towardsai.net/a-full-introduction-on-text-summarization-using-deep-learning-with-sample-code-ft-huggingface-d21e0336f50c) if you still have question.

Consider following me on Twitter ([@NLPiation](https://twitter.com/NLPiation)) where I am mostly writing about NLP and also is a great place to have discussions.